In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from tensorflow.keras import layers, models
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
import tensorflow as tf
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt


/bin/python3.11


2025-03-28 15:29:08.473273: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 15:29:08.486200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743172148.501042 2706650 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743172148.505551 2706650 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 15:29:08.521784: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
data_file = os.path.join(univariate_data_path, 'merged_univariate.npy')
data = np.load(data_file, allow_pickle=True)
print(data.shape)

# Check for NaN or Inf values in signals
for sample in data:
    if np.isnan(sample['signal']).any() or np.isinf(sample['signal']).any():
        print(f"NaN or Inf detected in {sample['record_name']}")

(666,)


In [4]:
# Masking function using patch-based masking
def mask_data(x, mask_ratio=0.5, patch_size=8):
    """ Apply patch-based masking to 1D signals """
    x_masked = np.copy(x)
    num_patches = x.shape[1] // patch_size  # Number of patches
    mask = np.random.rand(num_patches) < mask_ratio  # Randomly mask patches
    for i in range(num_patches):
        if mask[i]:
            x_masked[:, i * patch_size:(i + 1) * patch_size, :] = 0  # Zero out patches
    return x_masked

# Create windows from the data
def create_windows(sequence, window_size, step_size):
    windows = [sequence[i:i+window_size] for i in range(0, len(sequence) - window_size + 1, step_size)]
    return np.array(windows)

# Set windowing parameters
window_size = 500  
step_size = 250    

# Assuming `data` is a list of dictionaries with signal data
# Apply windowing to all records
all_windows = [create_windows(record['signal'], window_size, step_size) for record in data]
all_windows = np.concatenate(all_windows, axis=0)

# Reshape for Conv1D (batch_size, time_steps, channels)
all_windows = np.expand_dims(all_windows, axis=-1)  
print(f"Processed window shape: {all_windows.shape}")

Processed window shape: (454061, 500, 1)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

class FullyConvolutionalMaskedAutoencoder(nn.Module):
    def __init__(self, input_channels=1, num_layers=3, filters=[64, 32, 16]):
        super().__init__()
        self.encoder = self.build_encoder(input_channels, num_layers, filters)
        self.decoder = self.build_decoder(input_channels, num_layers, filters)
    
    def build_encoder(self, input_channels, num_layers, filters):
        layers = []
        in_channels = input_channels
        for i in range(num_layers):
            layers.append(nn.Conv1d(in_channels, filters[i], kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            if i < num_layers - 1:
                layers.append(nn.MaxPool1d(kernel_size=2, stride=2))
            in_channels = filters[i]
        return nn.Sequential(*layers)
    
    def build_decoder(self, input_channels, num_layers, filters):
        layers = []
        in_channels = filters[-1]
        for i in range(num_layers):
            layers.append(nn.ConvTranspose1d(in_channels, filters[-(i + 1)], kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            if i < num_layers - 1:
                layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
            in_channels = filters[-(i + 1)]
        layers.append(nn.ConvTranspose1d(in_channels, input_channels, kernel_size=3, padding=1))
        layers.append(nn.Sigmoid())
        return nn.Sequential(*layers)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def extract_encoder(self):
        return self.encoder  # Returns only the encoder part


# Define training loop
def train_fcmae(model, train_data, val_data, epochs=50, batch_size=32, learning_rate=1e-4, device='cuda'):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")
    
    return model

# Prepare data
X_train, X_val = train_test_split(all_windows, test_size=0.2, random_state=42)
X_train_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_train])
X_val_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_val])

# Convert to PyTorch tensors
tensor_X_train = torch.tensor(X_train_masked, dtype=torch.float32).permute(0, 2, 1)  # [batch, channels, seq_len]
tensor_X_val = torch.tensor(X_val_masked, dtype=torch.float32).permute(0, 2, 1)
tensor_Y_train = torch.tensor(X_train, dtype=torch.float32).permute(0, 2, 1)
tensor_Y_val = torch.tensor(X_val, dtype=torch.float32).permute(0, 2, 1)


# Create datasets & loaders
train_dataset = TensorDataset(tensor_X_train, tensor_Y_train)
val_dataset = TensorDataset(tensor_X_val, tensor_Y_val)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = FullyConvolutionalMaskedAutoencoder(input_channels=1, num_layers=3, filters=[64, 32, 16])
trained_model = train_fcmae(autoencoder, train_dataset, val_dataset, epochs=1, batch_size=32, device=device)

# Extract & save encoder
encoder = autoencoder.extract_encoder()
save_location = os.path.join(models_path, 'encoder_model.pth')
torch.save(encoder.state_dict(), save_location)
print("Saved encoder model!")


Epoch 1/1, Train Loss: 8.807820767206236, Val Loss: 8.608449170419748
Saved encoder model!
